# Time Zones

> Lasciate ogne speranza, voi ch'intrate (Abandon all hope, ye who enter here) —Dante Alighieri

## The Nature of Time

Getting datetimes **entirely** right in the face of timezones is surprisingly difficult, and the more effort you put into it, the more difficulty you discover.  In this lesson, we will make a first approximation, and a second approximation, toward getting the general problem of timezones right.  Subsequent approximations are outside the scope of this course.

In [1]:
from datetime import timezone, datetime, timedelta
from dateutil import tz
start = datetime.now()

## First Pass

The Python standard library module, `datetime` has rudimentary handling of timezones, and provides a framework for the third party tool we discuss in this lesson.  In this lesson, we will only look at datetime objects, which are the only ones with a concept of timezone.

Suppose we have several geographic locations in our organization, and they record events that occur at their location.  The record they make—this might be a server log file, or it might be a human action, for example—shows the local time when something occurred.

Being able to ask questions like whether one event occurred before another, or what was the time duration between these events is often relevant.  Thank about bank or trading transactions, for example; or of latency in computer servers being monitored.

We can define time zones where these local offices are operating and recording.

In [2]:
katmandu = timezone(timedelta(hours=5, minutes=45), name="Nepal")
havana = timezone(timedelta(hours=-5), name="Cuba")
newyork = timezone(timedelta(hours=-5), name="US-East")
nome = timezone(timedelta(hours=-9), name="US-Artic")

for server in [katmandu, havana, newyork, nome]:
    print(f"{str(server):>10} | {server.utcoffset(start)}")

     Nepal | 5:45:00
      Cuba | -1 day, 19:00:00
   US-East | -1 day, 19:00:00
  US-Artic | -1 day, 15:00:00


Let us record several events that we might want to process, about activities of Santa Claus and his elves.  Notice that now when we create datetimes, we add the optional `tzinfo` field with a timezone object in each.

In [3]:
elves = {"Give Hartaj train set":
             datetime(2020, 12, 24, 13, 30, 45, tzinfo=katmandu),
         "Build a train set":
             datetime(2020, 12, 23, 22, 35, 45, tzinfo=nome),
         "Bagels for Spring Party":
             datetime(2020, 3, 7, 14, 0, 0, tzinfo=newyork),
         "Rum for Spring Party":
             datetime(2020, 3, 8, 0, 30, 0, tzinfo=havana),
         "Lox for Spring Party":
             datetime(2020, 3, 8, 0, 30, 0, tzinfo=newyork),
         "Back for Pineapple":
             datetime(2020, 3, 8, 14, 0, 0, tzinfo=havana),
        }

Santa's workshop would like to examine how efficient his magic elves are in their "build on demand" schedule for toys, but the even times in various time zones make it less than immediately apparent.

In [4]:
build_on_demand = (elves['Give Hartaj train set'] 
                     - elves['Build a train set'])
print(build_on_demand)

0:10:00


Not bad, managing to build the train then deliver it to Katmandu in 10 minutes.  Magic powers definitely aid in the Taylorist supply chain.

We might also want to compare events in different time zones for equality or inequality.

In [5]:
print(elves['Rum for Spring Party'].tzinfo, "/",
      elves['Lox for Spring Party'].tzinfo)
elves['Rum for Spring Party'] == elves['Lox for Spring Party']

Cuba / US-East


True

In [6]:
elves["Bagels for Spring Party"] < elves["Rum for Spring Party"]

True

We can ask of any datetime what its offset from Coordinated Universal Time (UTC) is.

In [7]:
bagels = elves['Bagels for Spring Party']
print(bagels.tzname(), datetime.utcoffset(bagels))

US-East -1 day, 19:00:00


In [8]:
build = elves['Build a train set']
print(build.tzname(), datetime.utcoffset(build))

US-Artic -1 day, 15:00:00


## Second Pass

The simplified time zones defined using standard Python handle a fixed offset from UTC.  Actual time zones are much more complicated though.  Here we will use the third-party module `dateutil` to access the *IANA time zone database*—also called *tzdata*, the *zoneinfo database*, or *Olson database*—to get more sophisticated `timezone` objects.

At the time I am writing, Python 3.9 is in release candidate 1.  When it 3.9 arrives, it will contain a new standard library module called `zoneinfo` which will provide access to the IANA time zone database.  However, `dateutil` will remain a useful module, since it contains numerous other capabilities; it is discussed more in the next lesson.

### History

Some arcane difficulties arise with historical changes to calendrics.  If you are only concerned with working with "modern" datetimes from the last few decades, much of this has been standardized and officially documented.  In the last lesson, I passingly mentioned the transition between Gregorian and Julian calendar in England and its colonies, in 1752.  Most of Europe transitioned a century or two earlier; but Russia, for example, only did so in 1918, and therefore has a analagous gap of February 1-13, 1918 not existing there.

Or in another similar footnote, between 1155 and 1752 CE, England celebrated its new year on March 25 (Lady Day/Feast of the Annunciation), rather than on January 1.  Which is to say, for example, one day after Walter Raleigh was given a charter to colonize Virginia, in 1584, the date was March 26, 1585.  Automating caclulations of historical durations is tricky.

In a perfect world, some sophisticated database or marker of the meaning of a date and time at a particular date, time, and place, might work out all these special rules for us. I do not know of any software that exists that does that.

In [9]:
soviet = None  # Placeholder, not actually doing this right
olddays = datetime(1918, 1, 31, tzinfo=soviet)
newdays = datetime(1918, 2, 14, tzinfo=soviet)

In [10]:
fmt = "%B %d, %G"
print(" Old:", datetime.strftime(olddays, fmt))
print(" New:", datetime.strftime(newdays, fmt))
print("Diff:", newdays - olddays)
print("Wish: SHOULD BE 1 day")

 Old: January 31, 1918
 New: February 14, 1918
Diff: 14 days, 0:00:00
Wish: SHOULD BE 1 day


### Since 1970

Let us assume we are only worried about datetimes after 1970.  The IANA timezone database covers time zones, and their changes back to the start of Unix time.  Even here there are complications.  Often using the database will do the right thing; but exceptions remain.

Different nations, or other jurisdictions (such as different US states), follow different rules about Daylight Savings Time (DST).  Whether a time change of an hour (or occasionally two hours) occured between two datetimes depends, in part, on whether DST went into or came out of effect, in either or both timezone-sensitive objects.  

Notably, the southern and northern hemispheres have oppositive seasonality, so locations in each typically jump in opposite directions around similar dates.  Of course, many different specific times for a DST change exist legislatively in different jurisdictions, if that system is used at all in a jurisdiction.  

Let us define several time zones using the IANA timezone database.  Notice that when we ask for name or offset, it is specific to the datetime asked about.  For example, New York is either EST or EDT, depending on the time of year.

In [11]:
katmandu = tz.gettz("Asia/Kathmandu")
havana = tz.gettz("America/Havana")
newyork = tz.gettz("America/New_York")
nome = tz.gettz("America/Nome")

for server in [katmandu, havana, newyork, nome]:
    print(f"{str(server):>47} | "
          f"{server.tzname(start):>5} | "
          f"{server.utcoffset(start)}")

   tzfile('/usr/share/zoneinfo/Asia/Kathmandu') | +0545 | 5:45:00
   tzfile('/usr/share/zoneinfo/America/Havana') |   CDT | -1 day, 20:00:00
 tzfile('/usr/share/zoneinfo/America/New_York') |   EDT | -1 day, 20:00:00
     tzfile('/usr/share/zoneinfo/America/Nome') |  AKDT | -1 day, 16:00:00


With our more nuanced timezones available, let us reconfigure our events to use these improved time zones.  The setup is identical, but the timezone objects are no longer the simple offsets from UTC.

In [12]:
elves2 = {"Give Hartaj train set":
              datetime(2020, 12, 24, 13, 30, 45, tzinfo=katmandu),
          "Build a train set":
              datetime(2020, 12, 23, 22, 35, 45, tzinfo=nome),
          "Bagels for Spring Party":
              datetime(2020, 3, 7, 14, 0, 0, tzinfo=newyork),
          "Rum for Spring Party":
              datetime(2020, 3, 8, 0, 30, 0, tzinfo=havana),
          "Lox for Spring Party":
              datetime(2020, 3, 8, 0, 30, 0, tzinfo=newyork),
          "Back for Pineapple":
              datetime(2020, 3, 8, 14, 0, 0, tzinfo=havana),
         }

We can ask the same questions we did before.  For example, this does not change the "build on demand" duration for magic elves.

In [13]:
build_on_demand = (elves2['Give Hartaj train set'] 
                     - elves2['Build a train set'])
print(build_on_demand)

0:10:00


Other questions give different answers.  Remember that in naive time zones, the elves got rum in Havana and lox in New York at the very same moment.  Something funny happens when we use the database.

In [14]:
print("Havana", elves2['Back for Pineapple'])
print("Havana", elves2['Rum for Spring Party'])
# First purchase 12:30 am local time
print(elves2['Back for Pineapple'] - elves2['Rum for Spring Party'])

Havana 2020-03-08 14:00:00-04:00
Havana 2020-03-08 00:30:00-04:00
13:30:00


In [15]:
print("Havana  ", elves2['Back for Pineapple'])
print("New York", elves2['Lox for Spring Party'])
# First purchase 12:30 am local time
print(elves2['Back for Pineapple'] - elves2['Lox for Spring Party'])

Havana   2020-03-08 14:00:00-04:00
New York 2020-03-08 00:30:00-05:00
12:30:00


That was strange! Surely 12:30 am to 14:00 is 13.5 hours.  The problem is that on March 8, daylight savings time jumps ahead by an hour.

The further problem is that it does so at midnight in Havana, but at 2 am in New York.  Moreover, when the clock jumps ahead, some period of time simply does not exist (much as some dates did not exist historically, but in a place dependent way).

In [16]:
for event in ['Bagels for Spring Party', 'Rum for Spring Party',
              'Lox for Spring Party', 'Back for Pineapple']:
    dt = elves2[event]
    print(f"{event:>25} | datetime exists? {tz.datetime_exists(dt)}")

  Bagels for Spring Party | datetime exists? True
     Rum for Spring Party | datetime exists? False
     Lox for Spring Party | datetime exists? True
       Back for Pineapple | datetime exists? True


There is the problem.  The time the elf recorded for picking up rum in Havana never existed at all as a local time.  Someone is cooking the books!

Let us think about another duration.  How long is it from March 7 at 2 pm in New York, until March 8 at 2 pm in Havana (at least in 2020)?

In [17]:
print(elves2['Back for Pineapple'] 
        - elves2['Bagels for Spring Party'])

23:00:00


These datetimes are, naturally, 23 hours apart.  Both locations had their jump forward for daylight savings, at some point between those two datetimes.  Of course, sometimes DST moves in the other direction.  In that case, it is not that some datetimes are skipped, but rather that some occur twice.  In that case, some datetimes occur twice.  We can ask about that ambiguity.

In [18]:
back1 = datetime(2020, 11, 1, 1, 30, 0, tzinfo=newyork)
back2 = datetime(2020, 11, 1, 1, 31, 0, tzinfo=newyork)
later = datetime(2020, 11, 1, 3, 0, 0, tzinfo=newyork)
for dt in [back1, back2, later]:
    print(dt, "| ambiguous?", tz.datetime_ambiguous(dt))

2020-11-01 01:30:00-04:00 | ambiguous? True
2020-11-01 01:31:00-04:00 | ambiguous? True
2020-11-01 03:00:00-05:00 | ambiguous? False


We could *try* to make comparisons of these ambiguous datetimes, but the results do not give us great confidence.  Those two datetimes *might be* a minute apart, but we really do not know.

In [19]:
back2 - back1

datetime.timedelta(seconds=60)

We can explicitly disambiguate by specifying the argument `fold` that says whether we want the first or second version of an ambiguous datetime.  Unfortunately, this only gives us a marker of the time repetition, it does not do the appropriate arithmetic for us.  By default, a datetime assumes it is unfolded (earlier).

In [20]:
back3 = datetime(2020, 11, 1, 1, 30, 0, tzinfo=newyork, fold=False)
back4 = datetime(2020, 11, 1, 1, 31, 0, tzinfo=newyork, fold=True)
print(back4 - back3)

back5 = datetime(2020, 11, 1, 1, 30, 0, tzinfo=newyork, fold=True)
back6 = datetime(2020, 11, 1, 1, 31, 0, tzinfo=newyork, fold=False)
print(back6 - back5)

back7 = datetime(2020, 11, 1, 1, 30, 0, tzinfo=newyork, fold=True)
back8 = datetime(2020, 11, 1, 1, 31, 0, tzinfo=newyork, fold=True)
print(back8 - back7)

0:01:00
0:01:00
0:01:00


Let us do better, slightly by hand.

In [26]:
def fold_compare(dt1, dt2):
    "How long from dt1 until dt2? (possibly negative)"
    delta = timedelta(hours=dt2.fold) - timedelta(hours=dt1.fold) 
    return (dt2 - dt1) + delta

print("Fold none:", fold_compare(back1, back2))
print(" Fold 2nd:", fold_compare(back3, back4))
print(" Fold 1st:", fold_compare(back5, back6))
print("Fold both:", fold_compare(back7, back8))

Fold none: 0:01:00
 Fold 2nd: 1:01:00
 Fold 1st: -1 day, 23:01:00
Fold both: 0:01:00


## Down the rabbit hole...

Often, using the IANA timezone database, either via the `dateutil` module, the future standard library `zoneinfo`, or using another module like `pytz` that references the same database, will do the right thing.  *Often*. **Not always**.

One problem is that leap seconds are simply ignored by `datetime`.  The thing to understand is that a "second" in UTC—and in all the timezones that are defined by offsets from UTC—is **not a measure of time**.  It is a measure of the angular rotation of the Earth.  Specifically, it is approximately 15.041 seconds of angular longitude (relative to the sun).  This odd number comes from the fact that Earth needs to rotate slightly more than 360° to return to facing the sun because of simultaneos prograde revolution around the sun.

Unfortunately, the speed of the rotation of the Earth is not constant; rather it varies slightly because of plate tectonics, magma flow, and the tidal locking of the moon's orbit.  Hence, the International Earth Rotation and Reference Systems Service (IERS) adds (or possibly subtracts, though that has not occurred since 1970) a second to UTC from time to time.  This happens slightly less often than once a year, with a few months warning.

Here is an example of a leap second that `datetime`, and in fact also the IANA timezone database, simply ignores.

In [22]:
before = datetime(2008, 12, 31, 18, 59, 59, tzinfo=tz.gettz('EST'))
after = datetime(2008, 12, 31, 19, 0, 0, tzinfo=tz.gettz('EST'))
print("SHOULD BE 2 seconds")
after - before

SHOULD BE 2 seconds


datetime.timedelta(seconds=1)

Not only does Python's `datetime` not know how to handle those leap seconds, your operating system and other applications do not know how to handle them.  "Unix time" is also not a measure of time, but rather builds in the assumption that a day consists of 86,400 seconds, whether or not a particular day actually has one more or one less second according to UTC.  This is wrong about once a year.  Windows, MacOS, iOS, and Android behave the same as Linux and other Unix-like systems, in this regard.

What actually happens is that your computer checks a Network Time Protocol (NTP) server fairly often, when it is online, and after leap second events, the NTP server tells your computer that it is a second off.  These checks happen other times as well, of course.  Your computer quite likely adjusts its internal clock by a few microseconds every day, even in the absence of leap seconds.

### Does it Matter?

If you are trying to remember your family's birthdays or set a wake-up alarm, these leap seconds do not matter.  However, if you are trying to coordinate computer servers, they quite likely do.  For example, if your database has transactions with commit and rollback capabilities (like nearly all RDBMSs do), knowing the actual order of commands that might be less than a second apart is crucial.

### Descending Further

Even some of the things that the timezone database *should* handle, it seems not to; at least not as exposed in Python.  For example, Samoa and Tokelau decided in 2011 to switch to the other side of the international dateline. As a consequence:

In [25]:
# Capital city Apia cannonical TZ description (Samoa is alias)
before = datetime(2011, 12, 29, 23, 59, 0, tzinfo=tz.gettz('Pacific/Apia'))
after = datetime(2011, 12, 31, 0, 0, 0, tzinfo=tz.gettz('Pacific/Apia'))

print("SHOULD BE 1 minute")
print(after - before)

SHOULD BE 1 minute
1 day, 0:01:00


It would be correct if the database handled this as a permanent 24 hour "daylight savings time" jump forward.

The last section of this lesson is mostly warnings about issues that you will, hopefully, not face.  Few of them have anything to do with Python per se, but are general world time issues.

If you are fortunate enough to control the creation of timestamps by the various systems, it is always easiest to canonicalize everything to UTC before recording any data at all.  That option does not always exist, however.

Having all systems use UTC avoids a number of problems, but does not really address the leap second issue.  Special systems deal with this in various ways.  Google, for example, who obviously have a huge number of servers under their control, have implemented a special "leap smear" in their NTP.

Rather than add a leap second all at once, the Google servers (and others) keep 60 second minutes throughout, but make each second over the course of a period of time (e.g. day) a few microseconds longer than it would be otherwise.  This makes time monotonic and continuous, but different from the rest of the world during that day.